## Comparison for LogSumExp

### N=100

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [9]:
import numpy as np
from TestFunctions import LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers.GradientMethods import FGM_internal
import comparison
import imp
from comparison import method_comparison, create_methods_dict

In [10]:
N = 100
M = 2
alpha = 1
beta = 0.001
B = np.random.uniform(low = -1, high = 1, size = (M, N))
c = np.random.uniform(low = 0, high = 1, size = (M, ))

In [11]:
x_ = np.zeros((N,))
f_ = LogSumExp.r(alpha, beta).get_value(x_)
gamma = c.max()
y_max = f_/gamma
Q = [[0, y_max] for i in range(M)]
Q = np.array(Q)
size_domain = np.linalg.norm(Q[:,0]-Q[:,1])
Q[1:]

array([[0.        , 7.48206389]])

In [12]:
LSE = TestFunction(r = LogSumExp.h(c, size_domain = size_domain),
                F = LogSumExp.F_tilde(B, size_domain),
                h = LogSumExp.r(alpha, beta, size_domain),
                solver = FGM_internal, 
                get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))

In [14]:
history = {}
eps = 1e-4
methods = create_methods_dict(LSE, Q[:,1], size_domain, Q, eps, history, time_max = 5)

In [15]:
imp.reload(comparison)
comparison.method_comparison(methods)

In [17]:
history

{'FGM': [(array([7.48206389, 7.48206389]), 1587228122.7124193),
  (array([0., 0.]), 1587228122.9659564),
  (array([-0.0004992,  0.       ]), 1587228123.1084385),
  (array([-0.0002247,  0.       ]), 1587228123.157401),
  (array([-0.00037564,  0.        ]), 1587228123.2469025),
  (array([-0.00029264,  0.        ]), 1587228123.3636262),
  (array([-0.00033828,  0.        ]), 1587228123.4661527),
  (array([-0.00031318,  0.        ]), 1587228123.721404)],
 'Ellipsoids': [array([3.74103194, 3.74103194]),
  (array([2.40024231, 2.59545162]), 1587228122.9761581)],
 'Dichotomy': [(array([3.74103194, 3.74103194]), 1587228122.7293599),
  (array([3.74103194, 3.74103194]), 1587228123.075796),
  (array([1.87051597, 3.74103194]), 1587228123.5865676),
  (array([1.87051597, 1.87051597]), 1587228123.791514)]}